In [1]:
import numpy as np
import decimal
from collections import OrderedDict
import time
import pprint

"""
Test parameters:
Lambda = 1.55e-6
n_0 = 1.52
n_1 = 1.55
d = 1.5e-6
a = 1.5e-6
p_q = (5,5)
step_size = 1
Result in seconds:
1.864, 1.839, 1.873
"""



#Defining variables
Lambda = 1.55e-6                      #Wavelength of the light in m
n_0 = 1.52                           #Cladding refractive index. With 1.53, 1.54 weird result is obtained (1.544)
n_1 = 1.55                            #Core refractive index
d = 1.5e-6                            #Half Thickness of waveguide in metres
a = 1.5e-6                            #Half width of waveguide in metres
p_q = (5,5)                           #Tuple that will search for modes p and q up to the specified numbers
step_size = 1                         #Step size for the iterations when finding the solution
k_x = 0                               #Transverse wavenumbers
k_y = 0                               #Transverse wavenumbers
gamma_x = 0                           #Transverse wavenumbers
gamma_y = 0                           #Transverse wavenumbers
beta = 0                              #Propagation constant
phi = 0                               #Optical phase
psi = 0                               #Optical phase
mu_0 = (4*np.pi)*1e-7                 #Permeability of free space
epsilon_0 = 8.854e-12                 #Permittivity of free space

#Start timing
start = time.time()

#Calculating preliminary parameters
k = 2*np.pi / Lambda                  #Wavenumber or circular wavenumber
w_ang = 2.998e8 * k
p_modes = np.resize(np.arange(1,p_q[0]+1,1),(p_q[0],1))
p_modes.resize(p_q[0],1)
q_modes = np.resize(np.arange(1,p_q[1]+1,1),(p_q[1],1))
q_modes.resize(p_q[1],1)

#Solution for E^x_pq
#Transverse wavenumbers calculation
solution_x = OrderedDict()
solution_y = OrderedDict()
range_vector = np.arange(0,int(np.sqrt(k**2*(n_1**2-n_0**2)))+1,step_size)
range_vector.resize(1,int(np.sqrt(k**2*(n_1**2-n_0**2)))+1)

#Wavenumbers in x
gamma_x = np.zeros((p_q[0],int(np.sqrt(k**2*(n_1**2-n_0**2)))+1), dtype = np.float64)
gamma_x = range_vector+gamma_x
k_x = np.sqrt(k**2*(n_1**2-n_0**2)-gamma_x**2)
upperpart = gamma_x * n_1**2
lowerpart = k_x * n_0**2

solution_x = OrderedDict()
for n in range(len(gamma_x)): 
    wz = np.absolute(n*np.pi*0.5+np.arctan(upperpart[n]/lowerpart[n])-k_x*a)
    if np.argmin(wz) != 0:
        solution_x[str(n+1)]={'gamma_x':gamma_x[n][np.argmin(wz)],'k_x':k_x[n][np.argmin(wz)]}

#Wavenumbers in y
gamma_y = np.zeros((p_q[1],int(np.sqrt(k**2*(n_1**2-n_0**2)))+1), dtype = np.float64)
gamma_y = range_vector+gamma_y
k_y = np.sqrt(k**2*(n_1**2-n_0**2)-gamma_y**2)

solution_y = OrderedDict()
for n in range(len(gamma_y)): 
    x = np.absolute(n*np.pi*0.5+np.arctan(gamma_y[n]/k_y[n])-k_y*d)
    if np.argmin(x) != 0:
        solution_y[str(n+1)]={'gamma_y':gamma_y[n][np.argmin(x)],'k_y':k_y[n][np.argmin(x)]}

#Combining the solutions:
solution_xy = OrderedDict()

for p,valx in solution_x.items():
    for q,valy in solution_y.items():
        beta = np.sqrt(k**2*n_1**2-(valx['k_x']**2+valy['k_y']**2))
        phi = (int(p)-1)*np.pi*0.5
        psi = (int(q)-1)*np.pi*0.5
        solution_xy[(int(p),int(q))] = {'k':k, 'beta':beta,'gamma_x':valx['gamma_x'], 'k_x':valx['k_x'],
                                              'gamma_y':valy['gamma_y'], 'k_y':valy['k_y'],'phi':phi, 'psi':psi}
        
#End timing
end = time.time()

#Print results
print(str(len(solution_xy.keys()))+' modes were found in {} seconds'.format(end-start))
print(pprint.pformat(solution_xy))

4 modes were found in 1.7660527229309082 seconds
OrderedDict([((1, 1),
              {'beta': 6211438.6806267854,
               'gamma_x': 1029742.0,
               'gamma_y': 1034408.0,
               'k': 4053667.940115862,
               'k_x': 673081.88397505484,
               'k_y': 665888.80350656831,
               'phi': 0.0,
               'psi': 0.0}),
             ((1, 2),
              {'beta': 6130962.9113921914,
               'gamma_x': 1029742.0,
               'gamma_y': 277012.0,
               'k': 4053667.940115862,
               'k_x': 673081.88397505484,
               'k_y': 1198612.5983633783,
               'phi': 0.0,
               'psi': 1.5707963267948966}),
             ((2, 1),
              {'beta': 6131425.4695840199,
               'gamma_x': 269771.0,
               'gamma_y': 1034408.0,
               'k': 4053667.940115862,
               'k_x': 1200263.0614404532,
               'k_y': 665888.80350656831,
               'phi': 1.5707963267948966

In [78]:
#Calculating the electric fields
#Additional Inputs

region_1 = OrderedDict()
region_2 = OrderedDict()
region_3 = OrderedDict()

for p_q,values in solution_xy.items():
    #print('This is p_q:'+str(p_q))
    #print('This is values:'+str(values))
    A = 1  #This needs to be calculated
    k_x = values['k_x']
    gamma_x = values['gamma_x']
    k_y = values['k_y']
    gamma_y = values['gamma_y']
    phi = values['phi']
    psi = values['psi']
    beta = values['beta']
    
    #Magnetic and electric fields:
    #Region 1
    x = np.linspace(-a,a,200)
    y = np.linspace(-d,d,200)
    H_y1 = A * np.cos(k_x*x_1-phi) * np.cos(k_y*y_1-psi) #Hy in region 1
    dH_y1x = -A * k_x * np.sin(k_x*x_1-phi) * np.cos(k_y*y_1-psi) #Partial derivative with respect x
    dH_y1xx = -A * k_x**2 * np.cos(k_x*x_1-phi) * np.cos(k_y*y_1-psi) #second order partial derivative with respect x
    dH_y1xy = A * k_x * k_y * np.sin(k_x*x_1-phi) * np.sin(k_y*y_1-psi) #Partial der. with respect x and with respect y
    dH_y1y = -A * k_y * np.cos(k_x*x_1-phi) * np.sin(k_y*y_1-psi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y1y
    E_x = ((w_ang * mu_0)/beta)*H_y1 + (1/(epsilon_0*w_ang*n_1**2*beta))*dH_y1xx
    E_y = (1/(epsilon_0*w_ang*n_1**2*beta)) * dH_y1xy
    E_z = (-1/(epsilon_0*w_ang*n_1**2)) * dH_y1x
    region_1[p_q] = {'x':x, 'y_1':y_1, 'H_x':H_x, 'H_y':H_y1, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z}

    #Region 2
    x = np.linspace(a,2*a,100)
    y = np.linspace(d,2*d,100)
    H_y2 = A * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.cos(k_y*y_2-psi) #Hy in region 2
    dH_y2x = -A * gamma_x * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.cos(k_y*y_2-psi) #Partial derivative with respect x
    dH_y2xx = -A * gamma_x**2 * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.cos(k_y*y_2-psi) #second order partial derivative with respect x
    dH_y2xy = A * gamma_x * k_y * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.sin(k_y*y_2-psi) #Partial der. with respect x and with respect y
    dH_y2y = -A * k_y * np.cos(k_x*a-phi) * np.exp(-gamma_x*(x_2-a)) * np.sin(k_y*y_2-psi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y2y
    E_x = ((w_ang * mu_0)/beta)*H_y2 + (1/(epsilon_0*w_ang*n_0**2*beta))*dH_y2xx
    E_y = (1/(epsilon_0*w_ang*n_0**2*beta)) * dH_y2xy
    E_z = (-1/(epsilon_0*w_ang*n_0**2)) * dH_y2x
    region_2[p_q] = {'x':x, 'y':y, 'H_x':H_x, 'H_y':H_y2, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z}

    #Region 3
    x = np.linspace(a,2*a,100)
    y = np.linspace(d,2*d,100)
    H_y3 = A * np.cos(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-psi) #Hy in region 3
    dH_y3x = -A * k_x * np.sin(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-psi)  #Partial derivative with respect x
    dH_y3xx = -A * k_x**2 * np.cos(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-phi) #second order partial derivative with respect x
    dH_y3xy = A * k_x * gamma_y * np.sin(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-psi) #Partial der. with respect x and with respect y
    dH_y3y = -A * gamma_y * np.cos(k_x*x_2-phi) * np.exp(-gamma_y*(y_2-d)) * np.cos(k_y*d-phi) #Partial der. with respect y
    H_x = 0
    H_z = (-1/beta) * dH_y3y
    E_x = ((w_ang * mu_0)/beta)*H_y3 + (1/(epsilon_0*w_ang*n_0**2*beta))*dH_y3xx
    E_y = (1/(epsilon_0*w_ang*n_0**2*beta)) * dH_y3xy
    E_z = (-1/(epsilon_0*w_ang*n_0**2)) * dH_y3x
    region_3[p_q] = {'x':x, 'y':y, 'H_x':H_x, 'H_y':H_y3, 'H_z':H_z, 'E_x':E_x, 'E_y':E_y, 'E_z':E_z}

    

In [97]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(region_1[2,2]['E_x']*1e6, region_1[2,2]['E_y'], 'b', label = 'Region 1')
#plt.plot(region_2[2,1]['x']*1e6, region_2[2,1]['E_x'],'r', label = 'Region 2 right')
#plt.plot(region_2[2,1]['x']*-1e6, -region_2[2,1]['E_x'],'r', label = 'Region 2 left')
#plt.plot(region_2_left[1,1]['x_2']*1e6, region_2_left[1,1]['E_x'],'g', label = 'Region 2 left')
#plt.plot(region_2_right[1,1]['x_2']*-1e6, region_2_right[1,1]['E_x'], 'r', label = 'Region 2 right')
#plt.plot(region_2_left[1,1]['x_2']*1e6, region_2_left[1,1]['E_x'], 'g', label = 'Region 2 left')
#plt.plot(region_3_right[1,1]['y_2']*1e6, region_3_right[1,1]['E_y'], 'g', label = 'Region 3')
#plt.plot(region_3_right[1,1]['x_2']*1e6, region_3_right[1,1]['E_x'], 'g', label = 'Region 3')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc = 0)
plt.grid(True)
plt.show()

In [70]:
#3d plot
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.gca(projection='3d')

x, y = np.meshgrid(region_1[1,2]['E_x'], region_2[1,2]['E_x'])
z = 0
surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

plt.show()